# Background
---

# Prior and Simulations
---

In [2]:
from glass_sim import *

## 1. Prior Parameters: $\bm{\theta}\sim P(\bm{\theta})$

In [3]:
def prior_param_samples(h_range, Oc_range, Ob_range, type, n_samples):
    assert isinstance(h_range, tuple) and len(h_range) == 2, "h_range must be a tuple of (min, max)"
    assert isinstance(Oc_range, tuple) and len(Oc_range) == 2, "Oc_range must be a tuple of (min, max)"
    assert isinstance(Ob_range, tuple) and len(Ob_range) == 2, "Ob_range must be a tuple of (min, max)"
    assert type in ["uniform", "normal"], "type must be either 'uniform' or 'normal'"
    assert isinstance(n_samples, int) and n_samples > 0, "n_samples must be a positive integer"
    if type == "uniform":
        h_samples = np.random.uniform(h_range[0], h_range[1], n_samples)
        Oc_samples = np.random.uniform(Oc_range[0], Oc_range[1], n_samples)
        Ob_samples = np.random.uniform(Ob_range[0], Ob_range[1], n_samples)
        samples = np.vstack((h_samples, Oc_samples, Ob_samples)).T
    elif type == "normal":
        cov = np.diag([h_range[1] - h_range[0], Oc_range[1] - Oc_range[0], Ob_range[1] - Ob_range[0]])**2 / 12
        mean = [(h_range[0] + h_range[1]) / 2, (Oc_range[0] + Oc_range[1]) / 2, (Ob_range[0] + Ob_range[1]) / 2]
        samples = np.random.multivariate_normal(mean, cov, n_samples)
    return samples


h_range = (0.6, 0.8)
Oc_range = (0.2, 0.4)
Ob_range = (0.03, 0.05)
n_samples = 2
uniform_samples = prior_param_samples(h_range, Oc_range, Ob_range, "uniform", n_samples)
print("Uniform Samples:\n", uniform_samples)
normal_samples = prior_param_samples(h_range, Oc_range, Ob_range, "normal", n_samples)
print("Normal Samples:\n", normal_samples)


Uniform Samples:
 [[0.65667354 0.27998559 0.03819035]
 [0.67953725 0.29625809 0.046134  ]]
Normal Samples:
 [[0.78864674 0.36224045 0.03514576]
 [0.78369405 0.33553741 0.03775281]]


##  2. Simulated Data: $\bm{x}|\bm{\theta}$

In [10]:
param_samples = []
cls_samples = []

for i in range(uniform_samples.shape[0]):
    cosmology = {}
    cosmology['h'], cosmology['Oc'], cosmology['Ob'] = uniform_samples[i] # unpacking
    sim = lensing_cls_sim(cosmology)
    param_samples.append(sim['params'])
    cls_samples.append(sim['cls'])
param_samples = np.array(param_samples)
cls_samples = np.array(cls_samples) 

print(param_samples)
print(cls_samples)

17it [00:00, 35.93it/s]
16it [00:00, 36.90it/s]

[[0.65667354 0.27998559 0.03819035]
 [0.67953725 0.29625809 0.046134  ]]
[[4.43053525e-10 1.31514066e-08 4.02781425e-08 1.46934566e-08
  2.21749553e-08 4.38971984e-08 1.97120858e-08 1.63693117e-08
  3.56966750e-08 1.73967851e-08 1.88446427e-08 2.63635115e-08
  9.41838817e-09 1.86997813e-08 1.84491617e-08 2.10074818e-08
  2.24809962e-08 1.23631087e-08 1.29131299e-08 8.82358733e-09
  1.53017557e-08 1.85755540e-08 1.80402175e-08 1.38803814e-08
  7.80916231e-09 1.08815865e-08 1.19966738e-08 1.00551847e-08
  1.48924293e-08 1.02236750e-08 9.44799835e-09 6.81310361e-09
  6.71164443e-09 5.47961612e-09 6.48279693e-09 8.03249687e-09
  5.15785365e-09 8.11317259e-09 6.17755235e-09 6.84927155e-09
  7.54491381e-09 6.81444374e-09 4.99309606e-09 5.24038531e-09
  4.67574764e-09 6.43233357e-09 3.24859586e-09 5.59897068e-09
  4.62066530e-09 6.63155733e-09 4.87234830e-09 4.49208122e-09
  5.59395749e-09 4.27254741e-09 4.09752724e-09 4.43454889e-09
  5.23475048e-09 5.03346566e-09 3.76675043e-09 3.96166489e-

# Compression
---

## Three compression methods

1. Canoncical correlation analysis
    - Maximises the correlation between the parameters and data (also could be written as maximising the mutual information).
    - Use canonical data vectors as compressed data
1. MOPED
    - assumes Gaussian likelihood surface, as a result:
        - requires fiducial cosmology simulations for covariance estimation
        - requires additional simulation cost for mean derivative estimation
1. e-MOPED 
    - Park, M., Gatti, M., & Jain, B. (2025). Dimensionality reduction techniques for statistical inference in cosmology.
    - Define new data space as a transform of the parameter space centred at the mean parameter.


In [ ]:
def cca(param_samples, cls_samples):
# Compute the sampled parameter auto covariance, simulated data vector auto covariance and the parameter-data vector cross covariance
# Methodology as per Park, M., Gatti, M., & Jain, B. (2025). Dimensionality reduction techniques for statistical inference in cosmology.

    cov = np.cov(params_samples.T, cls_samples.T)
    params_count = param_samples.shape[1]
    cp = cov[:params_count,:params_count]
    cd = cov[params_count:,params_count:]
    cpd = cov[:params_count,params_count:]

    cl = cpd.T@np.linalg.inv(cp)@cpd

    canon_corr, canon_vecs = scipy.linalg.eigh(cd, cd - cl)
    return {'params': params_samples, 'compressed_cls': cla_samples@canon_vecs}

In [ ]:
def moped(param_samples, cls_samples, fiducial_cls, perturbed_cls):
    '''
    fiducial_cls used for covariance computation
    perturbed_cls used for mean derivative computation
    '''
    return {'params': params, 'compressed_cls': compressed_cls}

In [ ]:
def emoped(param_samples, cls_samples, fiducial_cls, perturbed_cls):
    '''
    fiducial_cls used for covariance computation
    perturbed_cls used for mean derivative computation
    '''
    return {'params': params, 'compressed_cls': compressed_cls}

# Density Estimation
---